# Data Cleaning

This notebook is used to clean and combine all datasources into a coherent dataset. 

We have three different data groups:
* Label & Patient Data
* Lab Data
* MRI Data

All of these have to be cleaned, selected and combined into one dataset.

## Imports

In [1]:
import pandas as pd
import datetime
import numpy as np
import re

# Data Cleaning and Selection Label & Patient-data
First we will clean the data for our labels and patients. This includes the basic cleaning and data type matching as well as looking at anomalies and define the output format. We will use the prepared 'no duplicate PID' Sheet with Labels from KSA.

In [2]:
print("Start Clean and Preprocessing patients-data")

Start Clean and Preprocessing patients-data


In [3]:
df_patients = pd.read_excel(r'../raw_data/Hypophysenpatienten.xlsx',sheet_name='no duplicate PID')
df_patients.head()

,%ID,Fall Nr.,Datum/Zeit,Modalität,Exam Code,Exam Name,Abteilung,Arbeitsplatz.Kürzel,Aufnahmeart,PID,...,OP Datum,Ausfälle post,Diagnose,Kategorie,Patient Alter,Zuweiser,AnforderungDatum,ÜberweiserIntern.Bereich,ÜberweiserIntern.Klinik,Gender
0,8548799,41324884,2021-12-21 14:58:00,M-MR,MNDSHYP,MR Hypophyse,MRN,MRI4,Amb,4858,...,NaT,NaN,NaN,NaN,76,Endo,2021-11-09 14:38:55.0000000,MUK,Endokrinologie,male
1,7786984,40619017,2019-06-12 10:16:00,M-MR,MNDSHYP,MR Hypophyse,MRN,MRI4,Amb,6144,...,NaT,NaN,NaN,NaN,63,Neurochir,2018-09-04 15:03:12.0000000,KCH,Neurochirurgie,male
2,8899566,41814658,2023-03-07 10:29:00,M-MR,MNDSHYP,MR Hypophyse,MRN,MRI2,Amb,7048,...,NaT,NaN,NaN,NaN,60,Augenklinik,2023-01-25 19:43:52.0000000,KCH,Augenklinik,male
3,8376664,41172058,2021-03-27 15:05:00,M-MR,MNDSHYP,MR Hypophyse,MRN,MRI4,Amb,7375,...,NaT,NaN,inaktiv,non-prolaktinom,55,Endo,2021-03-22 18:29:03.0000000,MUK,Endokrinologie,male
4,7430454,40279590,2017-09-12 11:05:00,M-MR,MNDSHYP,MR Hypophyse,MRN,MRI3,Amb,14433,...,NaT,keine,inaktiv,non-prolaktinom,77,Berkmann Sven,2016-10-18 14:12:19.0000000,KCH,Neurochirurgie,female


In [4]:
df_patients.tail()

,%ID,Fall Nr.,Datum/Zeit,Modalität,Exam Code,Exam Name,Abteilung,Arbeitsplatz.Kürzel,Aufnahmeart,PID,...,OP Datum,Ausfälle post,Diagnose,Kategorie,Patient Alter,Zuweiser,AnforderungDatum,ÜberweiserIntern.Bereich,ÜberweiserIntern.Klinik,Gender
521,8711158,41598956,2022-08-16 13:13:00,M-MR,MNDSHYP,MR Hypophyse,MRN,MRI3,Amb,300375656,...,2022-05-23,keine,inaktiv,non-prolaktinom,68,Hirntumorzen,2022-06-02 16:05:45.0000000,KCH,Hirntumorzentrum,female
522,8686428,41596179,2022-07-12 12:15:00,M-MR,MNDSHYP,MR Hypophyse,MRN,MRI2,Amb,300375717,...,NaT,NaN,NaN,NaN,69,Rudofsky Gottfried,2022-05-04 08:17:31.0000000,-,-,male
523,8760431,41664583,2022-10-27 11:54:00,M-MR,MNDSHYP,MR Hypophyse,MRN,MRI1,Amb,300387138,...,NaT,keine,prolaktinom,prolaktinom,72,Neurochir,2022-08-05 09:51:39.0000000,KCH,Neurochirurgie,male
524,8884823,41792671,2023-01-09 18:12:00,M-MR,MNDSHYP,MR Hypophyse,MRN,MRI3,Stat,300394756,...,NaT,NaN,NaN,NaN,56,421 Neurochirurgische Station,2023-01-09 17:37:29.0000000,-,-,male
525,8251904,40950091,2020-10-14 08:13:00,M-MR,MNDSHYP,MR Hypophyse,MRN,MRI3,Amb,580000175,...,NaT,NaN,NaN,NaN,33,Endo,2020-09-23 17:21:06.0000000,MUK,Endokrinologie,female


In [5]:
df_patients.columns

Index(['%ID', 'Fall Nr.', 'Datum/Zeit', 'Modalität', 'Exam Code', 'Exam Name',
       'Abteilung', 'Arbeitsplatz.Kürzel', 'Aufnahmeart', 'PID', 'Grösse',
       'Ausfälle prä', 'Prolaktin', 'IGF1', 'Cortisol', 'fT4',
       'weiteres Labor', 'Qualität', 'ED', 'OP Datum', 'Ausfälle post',
       'Diagnose', 'Kategorie', 'Patient Alter', 'Zuweiser',
       'AnforderungDatum', 'ÜberweiserIntern.Bereich',
       'ÜberweiserIntern.Klinik', 'Gender'],
      dtype='object')

## Basic Cleaning, Column Selection, Anomaly Correction and Format definition


### Column Selection
First we select only the columns which have value to our model or our analysis. Some columns are already renamed to make their content more intuitive.

In [6]:
# define needed columns
column_list = ['PID','Fall Nr.','%ID',"Datum/Zeit","Arbeitsplatz.Kürzel",'Grösse',
       'Ausfälle prä', 'Qualität', 'ED','OP Datum',
       'Diagnose', 'Kategorie', 'Patient Alter',
       'Prolaktin',"IGF1", 'Cortisol','fT4','weiteres Labor','Gender']
df_patients = df_patients[column_list]
# rename columns
df_patients= df_patients.rename(columns={"Fall Nr.": "Case_ID","PID": "Patient_ID",'%ID':"MRI_Case_ID",
                       "Datum/Zeit": "Date_Case","ED": "Entry_date", "OP Datum": "Operation_date",
                       "Arbeitsplatz.Kürzel":"ID_MRI_Machine","Grösse": "Adenoma_size","Qualität": "Label_Quality",
                       "Patient Alter":"Patient_age","Kategorie":"Category","Diagnose":"Diagnosis",
                       "Prolaktin":"Prolactin","weiteres Labor":"Lab_additional", 'Gender':"Patient_gender"})

### Check for Anomalies and correct them
There are some Anomalies mostly in the datetime columns (eg. Operation date before Entry Date). These are corrected or were corrected by the KSA after feedback from us. 

In [7]:
# rows where Entry Date is after Operationdate?
assert len(df_patients[df_patients['Operation_date'] < df_patients['Entry_date']][['Entry_date','Operation_date']]) ==0

### Data Type Definition
Now we check the column data-types and parse them into their resprective type if not already correct.


In [8]:
# make datetime values
df_patients["Date_Case"] = pd.to_datetime(df_patients["Date_Case"])
df_patients["Entry_date"] = pd.to_datetime(df_patients["Entry_date"])
df_patients["Operation_date"] = pd.to_datetime(df_patients["Operation_date"])

In [9]:
# set category data type in pandas, check datatypes
df_patients['ID_MRI_Machine'] = df_patients['ID_MRI_Machine'].astype('category')
df_patients['Adenoma_size'] = df_patients['Adenoma_size'].astype('category')
df_patients['Diagnosis'] = df_patients['Diagnosis'].astype('category')
df_patients['Category'] = df_patients['Category'].astype('category')
df_patients['Patient_gender'] = df_patients['Patient_gender'].astype('category')
df_patients.dtypes

Patient_ID                 int64
Case_ID                    int64
MRI_Case_ID                int64
Date_Case         datetime64[ns]
ID_MRI_Machine          category
Adenoma_size            category
Ausfälle prä              object
Label_Quality             object
Entry_date        datetime64[ns]
Operation_date    datetime64[ns]
Diagnosis               category
Category                category
Patient_age                int64
Prolactin                 object
IGF1                      object
Cortisol                  object
fT4                       object
Lab_additional            object
Patient_gender          category
dtype: object

### Check Duplicates
Check if a patient is duplicated.

In [10]:
# Patient ID Duplicate Check
assert len(df_patients[df_patients["Patient_ID"].duplicated()]) == 0

### Check Diagnosis

In [11]:
df_patients['Diagnosis'].unique()

[NaN, 'inaktiv', 'empty sella', 'inaktiv, rathke', 'normalbefund', ..., 'inaktiv, apoplektiform', 'apoplektiformes prolaktinom', 'inaktiv (cortico)', 'Mikro-/Akromegalie', 'keine']
Length: 41
Categories (40, object): ['Akromegalie', 'Akromegalie, gh', 'Hypophyseninfarkt', 'Mikro-/Akromegalie', ..., 'substituiert alle Achsen', 'supprimiertes prolaktin', 'teils inaktiv, intra und supraselläres zystis..., 'zystisch']

## One Hot Encode Categorical Values

To use and analyse the categorical data we need to one-hot encode them. This is done by splitting the comma separated strings into single strings and then create a one-hot-encoded column of each individual value. This column is then added to the original dataframe.

In [12]:
df_patients["Ausfälle prä"]= df_patients["Ausfälle prä"].str.replace(' ', '')
df_patients["Ausfälle prä"]= df_patients["Ausfälle prä"].str.lower()
# Split the 'Ausfälle prä' column into separate strings
df_patients['Ausfälle prä'] = df_patients['Ausfälle prä'].str.split(',')

# Create a set to store all unique disfunctions
unique_disfunctions = set()

# Iterate over the 'Ausfälle prä' column to gather unique disfunctions
for value in df_patients['Ausfälle prä']:
    if isinstance(value, list):
        unique_disfunctions.update(value)
    elif isinstance(value, str):
        unique_disfunctions.add(value)

# Iterate over the unique disfunctions and create one-hot encoded columns
for disfunction in unique_disfunctions:
    df_patients["Pre_OP_hormone_"+ disfunction] = df_patients['Ausfälle prä'].apply(lambda x: 1 if (isinstance(x, list) and disfunction in x) or (x == disfunction) else 0)
# drop the original 'Ausfälle prä' column
df_patients = df_patients.drop('Ausfälle prä', axis=1)

In [13]:
df_patients["Diagnosis"]= df_patients["Diagnosis"].str.replace(' ', '')
df_patients["Diagnosis"]= df_patients["Diagnosis"].str.lower()
# Split the 'Ausfälle prä' column into separate strings
df_patients['Diagnosis'] = df_patients['Diagnosis'].str.split(',')

# Create a set to store all unique disfunctions
unique_disfunctions = set()

# Iterate over the 'Ausfälle prä' column to gather unique disfunctions
for value in df_patients['Diagnosis']:
    if isinstance(value, list):
        unique_disfunctions.update(value)
    elif isinstance(value, str):
        unique_disfunctions.add(value)

# Iterate over the unique disfunctions and create one-hot encoded columns
for disfunction in unique_disfunctions:
    df_patients["Diagnosis_"+ disfunction] = df_patients['Diagnosis'].apply(lambda x: 1 if (isinstance(x, list) and disfunction in x) or (x == disfunction) else 0)
# drop the original 'Ausfälle prä' column
df_patients = df_patients.drop('Diagnosis', axis=1)

## Remove All NA and not needed Labels

In [14]:
# remove all labels which are not prolaktion or non-prolaktinom
df_patients = df_patients[df_patients['Category'].isin(['non-prolaktinom','prolaktinom'])]
assert len(df_patients['Category'].unique()) == 2

In [15]:
df_patients.to_csv(r'../raw_data/label_data.csv',index=False)

In [16]:
print("End Clean and Preprocessing patient data")

End Clean and Preprocessing patient data


# Data Cleaning and Selection MRI-data

Now we will clean all MRI's.

In [17]:
print("Start Clean and Preprocessing mri data")

Start Clean and Preprocessing mri data


### Column Selection
Only select the interesting columns for the mri's.

In [18]:
column_list_mri = ['PID','Fall Nr.',"Datum/Zeit","Arbeitsplatz.Kürzel"]

In [19]:
df_mri = pd.read_excel(r'../raw_data/Hypophysenpatienten.xlsx',sheet_name='w duplicates')
# select and rename columns
df_mri = df_mri[column_list_mri]
df_mri= df_mri.rename(columns={"Fall Nr.": "Case_ID","PID": "Patient_ID",
                       "Datum/Zeit": "Date_Case","Arbeitsplatz.Kürzel":"ID_MRI_Machine"})

In [20]:
df_mri.head()

,Patient_ID,Case_ID,Date_Case,ID_MRI_Machine
0,300146159,0041835743,2023-05-11 09:00:00,MRI3
1,762512,0041708812,2023-05-06 08:12:00,MRI3
2,365189,0041892695,2023-05-05 14:19:00,MRI3
3,543641,0041725372,2023-05-05 07:54:00,MRI2
4,300302329,0041843364,2023-05-02 12:04:00,MRI3


In [21]:
df_mri['Case_ID'] = df_mri['Case_ID'].replace('-','',regex=True)

In [22]:
df_mri['Case_ID']=df_mri['Case_ID'].astype(int)

## Remove same Day MRI and take only newest

In [23]:
df_mri_clean = df_mri.groupby(["Patient_ID","Case_ID"])[["Date_Case",'ID_MRI_Machine']].max().reset_index()

# if there are multiple
n_cases = len(df_mri_clean)
df_mri_clean = df_mri_clean.groupby(["Patient_ID","Case_ID"])[["Date_Case",'ID_MRI_Machine']].max().reset_index()
print(f"{n_cases-len(df_mri_clean)} Cases were deleted, because they were same-day duplicates.")

0 Cases were deleted, because they were same-day duplicates.


In [24]:
df_mri_clean.to_csv(r'../raw_data/mri_data.csv',index=False)

In [25]:
print("End Clean and Preprocessing mri data")

End Clean and Preprocessing mri data


# Data Cleaning and Selection Lab-data

Now the lab data from the explicit KSA export will be cleaned.

In [26]:
print("Start Clean and Preprocessing lab-data")

Start Clean and Preprocessing lab-data


## Read

In [27]:
lab_data = pd.read_excel("../raw_data/extract_pit.xlsx",
                         usecols=['PATIENT_NR','FALL_NR','Analyse-ID','Resultat','Datum_Resultat']).rename(
                             columns={"PATIENT_NR":"Patient_ID","FALL_NR":"Case_ID","Analyse-ID":"Lab_ID",})

In [28]:
lab_data.columns

Index(['Case_ID', 'Patient_ID', 'Lab_ID', 'Datum_Resultat', 'Resultat'], dtype='object')

### Clean and select Lab's
There is a multitude of labs in the export. We do not need all of them. 

In [29]:
# remove not needed labs
lab_data= lab_data[~lab_data['Lab_ID'].isin(['ABTEST','TBILHB'])].copy()

In [30]:
# rename labs which are integer based with a string name
lab_data['Lab_ID'] = lab_data['Lab_ID'].replace({20396:'IGF1',24382:'PROL',24384:'PROL',24383:'PROL'})

In [31]:
# replace some not used characters in the case ids
lab_data['Case_ID'] = lab_data['Case_ID'].replace('#','',regex=True)
lab_data['Case_ID'] = lab_data['Case_ID'].astype(int)

In [32]:
# clean result column
lab_data['Resultat'] = lab_data['Resultat'].replace(',','.',regex=True)
lab_data['Resultat'] = lab_data['Resultat'].replace('>','',regex=True)
lab_data['Resultat'] = lab_data['Resultat'].replace('<','',regex=True)
lab_data['Resultat'] = lab_data['Resultat'].replace('¬†','',regex=True)
lab_data['Resultat']= lab_data['Resultat'].astype(float)

In [33]:
# replace export anomalies 
ids = {'Ã¼': 'ü', 'Ã¤': 'ä', "Ã„":"Ä","√§":"ä"}

for column in lab_data.columns[lab_data.columns.isin(["Case_ID","Patient_ID","Datum_Resultat","Auftragsdatum"]) == False]:
    for old, new in ids.items():
        lab_data[column] = lab_data[column].replace(old, new, regex=False)

# clean the greather and less than characters with regex
clean_result = lambda result: re.sub(r'(?<!\d)\.', '', re.sub(r'[^\d.]', '', str(result))) #clean < zahl / > zahl / 1 A zahl
lab_data["Resultat"] = lab_data["Resultat"].apply(clean_result) 
# remove empty results
lab_data = lab_data[lab_data["Resultat"] != ""]
lab_data["Resultat"] = lab_data["Resultat"].astype(float)

In [34]:
# check if the datetime was correctly fixed
assert lab_data["Datum_Resultat"].min() > pd.to_datetime("1995-01-01")

In [35]:
# mean of results of same date
lab_data = lab_data.groupby(["Patient_ID","Lab_ID","Datum_Resultat"])["Resultat"].agg(['mean']).reset_index()

## Merge Cases with Patient Cases

In [36]:
lab_data = pd.merge(lab_data,df_mri_clean,on="Patient_ID",how = "right")
lab_data = lab_data[lab_data["Date_Case"] >= lab_data["Datum_Resultat"]].drop(columns="Date_Case")

In [37]:
# Compute newest date for each patient and analysis
max_dates = lab_data.groupby(['Patient_ID', "Lab_ID","Case_ID"])['Datum_Resultat'].max().reset_index()
# Merge with the original DataFrame to filter rows with minimum dates
lab_data = pd.merge(lab_data, max_dates, on=['Patient_ID', 'Lab_ID', 'Datum_Resultat',"Case_ID"])

In [38]:
# check for any duplicate Values
assert len(lab_data.loc[:,["Case_ID","Lab_ID"]].drop_duplicates()) == len(lab_data)

In [39]:
# make dataframe wide
lab_data = lab_data.pivot(index=["Patient_ID","Case_ID"],values = ['mean'], columns = ['Lab_ID'])
lab_data.columns = lab_data.columns.droplevel()
lab_data = lab_data.reset_index()

### Create LabData from label data

In [40]:
df_additional_lab = pd.read_csv(r'../raw_data/label_data.csv').rename(columns={'Cortisol':'COR60','fT4':'FT4','Prolactin':'PROL'})[['Patient_ID','Case_ID','COR60','FT4','PROL','IGF1','Lab_additional']]
df_additional_lab.columns
df_additional_lab = df_additional_lab.dropna(subset=['PROL','IGF1','COR60','FT4','Lab_additional'], how='all').reset_index(drop=True)

In [41]:
df_additional_lab.head()

,Patient_ID,Case_ID,COR60,FT4,PROL,IGF1,Lab_additional
0,17081,40573077,766,14.3,13.5ug/L,16.6nmol,NaN
1,36127,41579190,110,7.3,687mU/l,75.4ng/ml,Testo 0.3nmol/l
2,62907,40456508,535,NaN,750ug/l,NaN,rest fehlt
3,82016,40345339,707,14.1,19.6ug/l,16.2nmol/l,NaN
4,112374,40541632,334,11,381ug/l,15.9nmol,Testo 3.8nmol


In [42]:
df_additional_lab.tail()

,Patient_ID,Case_ID,COR60,FT4,PROL,IGF1,Lab_additional
33,300088600,41467577,NaN,NaN,NaN,NaN,labor intern
34,300228153,41707994,329,10.1,173mU/l,6.3nmol/l,Testo 14.1nmol/l
35,300291886,41169249,311 nmol/l,14.6 pmol/l,7.8 ug/l,208 ng/ml,NaN
36,300311713,41683224,NaN,NaN,NaN,NaN,nihil
37,300312446,41718174,271,8.4,743mU/l,20.2nmol/l,NaN


In [43]:
df_additional_lab['Lab_additional'] =df_additional_lab['Lab_additional'].fillna('')
for i in ['Test', 'LH','FSH']:
    df_additional_lab[i] = ''
    indices = df_additional_lab[df_additional_lab['Lab_additional'].str.contains(i)].index
    df_additional_lab.loc[indices,i]  = df_additional_lab.iloc[indices]['Lab_additional']

In [44]:
df_additional_lab= df_additional_lab.drop(columns=['Lab_additional'])
df_additional_lab= df_additional_lab.rename(columns={'Test':'TEST'})

#### Testosteron Cleaning

In [45]:
df_additional_lab['TEST'] = df_additional_lab['TEST'].replace(r' nmol/l','',regex=True)
df_additional_lab['TEST'] = df_additional_lab['TEST'].replace(r'nmol/l','',regex=True)
df_additional_lab['TEST'] = df_additional_lab['TEST'].replace(r'nmol','',regex=True)
df_additional_lab['TEST'] = df_additional_lab['TEST'].replace(r'Testo','',regex=True)
df_additional_lab['TEST'] = df_additional_lab['TEST'].replace(r'Test','',regex=True)
df_additional_lab['TEST'] = df_additional_lab['TEST'].replace(r',','.',regex=True)
df_additional_lab.loc[df_additional_lab['TEST'] == '', 'TEST'] = np.nan
df_additional_lab['TEST']= df_additional_lab['TEST'].astype(float)

#### LH Cleaning

In [46]:
df_additional_lab.loc[df_additional_lab['LH'] == '', 'LH'] = np.nan
df_additional_lab['LH'] = df_additional_lab['LH'].replace(r'FSH \d*U\/L,','',regex=True)
df_additional_lab['LH'] = df_additional_lab['LH'].replace(r'LH','',regex=True)
df_additional_lab['LH'] = df_additional_lab['LH'].replace(r'U/L','',regex=True)
df_additional_lab['LH']= df_additional_lab['LH'].astype(float)

#### FSH Cleaning

In [47]:
df_additional_lab['FSH'] = df_additional_lab['FSH'].replace(r'FSH','',regex=True)
df_additional_lab['FSH'] = df_additional_lab['FSH'].replace(r'U/L','',regex=True)
df_additional_lab['FSH'] = df_additional_lab['FSH'].replace(r', LH \d*','',regex=True)

df_additional_lab.loc[df_additional_lab['FSH'] == '', 'FSH'] = np.nan
df_additional_lab['FSH']= df_additional_lab['FSH'].astype(float)

#### Cortisol Cleaning 

In [48]:
df_additional_lab['COR60'] = df_additional_lab['COR60'].replace(r' nmol/l','',regex=True)
df_additional_lab['COR60']= df_additional_lab['COR60'].astype(float)

#### FT4 Cleaning

In [49]:
df_additional_lab['FT4'] = df_additional_lab['FT4'].replace(r' pmol/l','',regex=True)
df_additional_lab['FT4']= df_additional_lab['FT4'].astype(float)

#### Prolaktin Cleaning and Conversion

In [50]:
df_additional_lab["PROL"]= df_additional_lab["PROL"].str.replace("ug/L","ug/l")
df_additional_lab["PROL"]= df_additional_lab["PROL"].str.replace("mu/L","mU/l")


In [51]:
# get indices which need to be converted
# indices_to_divide = df_additional_lab.loc[df_additional_lab["PROL"].str.contains('mU/l'),'PROL'].index 
indices_to_divide = df_additional_lab[~df_additional_lab["PROL"].isna() & df_additional_lab['PROL'].str.contains('mU/l')].index 
# remove units and strings
df_additional_lab['PROL'] = df_additional_lab['PROL'].str.rstrip(r'mU/l')
df_additional_lab['PROL'] = df_additional_lab['PROL'].str.rstrip(r'ug/l')
df_additional_lab['PROL'] = df_additional_lab['PROL'].str.rstrip(r'ug/L')
df_additional_lab['PROL'] = df_additional_lab['PROL'].astype(float)
# mU/l -> ug/l (mU/l * 0.048)
df_additional_lab.loc[indices_to_divide,'PROL'] = df_additional_lab.loc[indices_to_divide,'PROL'] * 0.048 


#### IGF1 Cleaning and Conversion

In [52]:
# df_additional_lab["IGF1"]= df_additional_lab["IGF1"].str.replace("ug/L","ug/l")
df_additional_lab["IGF1"]= df_additional_lab["IGF1"].str.replace("ug/l","")
df_additional_lab["IGF1"]= df_additional_lab["IGF1"].str.replace(",",".")

# get indices which need to be converted
# indices_to_divide = df_additional_lab.loc[df_additional_lab["IGF1"].str.contains('ng/ml'),'IGF1'].index 
indices_to_divide = df_additional_lab[~df_additional_lab["IGF1"].isna() & df_additional_lab['IGF1'].str.contains('ng/ml')].index
# remove units and strings
df_additional_lab['IGF1'] = df_additional_lab['IGF1'].str.rstrip(r'ng/ml')
df_additional_lab['IGF1'] = df_additional_lab['IGF1'].str.rstrip(r'nmol')
df_additional_lab['IGF1'] = df_additional_lab['IGF1'].str.rstrip(r'nmol/l')
df_additional_lab['IGF1'] = df_additional_lab['IGF1'].astype(float)
# ng/ml -> nmol/l (ng/ml * 0.13)
df_additional_lab.loc[indices_to_divide,'IGF1'] = df_additional_lab.loc[indices_to_divide,'IGF1'] * 0.13


In [53]:
# combine additional lab and lab data
lab_data = lab_data.set_index(['Patient_ID','Case_ID']).combine_first(df_additional_lab.set_index(['Patient_ID','Case_ID'])).reset_index()

In [54]:
spar = round(lab_data[['COR60', 'FSH','LH', 'FT4', 'IGF1', 'LH', 'PROL','TEST']].isna().mean().mean(),3)
print(f"Sparsity of labordata: {spar} % (nur von Fällen mit Laborwerten)")
print(f"Von {len(df_mri_clean)-len(lab_data)} Fällen gibt es keine Laborwerte.")

Sparsity of labordata: 0.315 % (nur von Fällen mit Laborwerten)
Von 509 Fällen gibt es keine Laborwerte.


In [55]:
lab_data.to_csv(r'../raw_data/lab_data.csv',index=False)

In [56]:
print("End Clean and Preprocessing labor data")

End Clean and Preprocessing labor data


# Full Merge

In [57]:
# get mridata where an mri matches to with case and patient if of a lab
df_temp = pd.merge(df_mri_clean,lab_data,left_on=['Patient_ID','Case_ID'],right_on=['Patient_ID','Case_ID'])

In [59]:
# match this data with the patients data
full_merged = pd.merge(df_temp,df_patients[['Patient_ID','Category','Patient_gender','Patient_age','Adenoma_size']+
                                           list(df_patients.columns[df_patients.columns.str.contains('Pre')])+
                                           list(df_patients.columns[df_patients.columns.str.contains('Diagnosis_')])+
                                              ['Operation_date', 'Entry_date','MRI_Case_ID',]]
                                              ,how='inner',left_on=['Patient_ID'],right_on=['Patient_ID'])

In [60]:
full_merged = full_merged.sort_values('Patient_ID')

In [61]:
assert full_merged.duplicated(subset=['Patient_ID','Case_ID']).sum() == 0

In [62]:
full_merged.to_csv(r'../raw_data/data_full_merge.csv',index=False)